In [21]:
import pandas as pd

#### Ορίζω το **path** του excel αρχείου ΚΑΙ το **πλήθος** των sheets

In [22]:
# path και όνομα exel αρχείου
excel_path = 'xlsx/Greek_NRC_VAD_Lex.xlsx'
# πληθος sheets
sheets_nbr = 1

#### Διαβάζω τα sheets του excel και σώζω τα dfs τους στη λίστα sheetsL

In [23]:
sheetsL = []
for i in range(0, sheets_nbr):
    sheet_df = pd.read_excel(excel_path, sheet_name=i)
    sheetsL.append(sheet_df)
    print('sheet1:', len(sheet_df), 'rows')

sheet1: 19971 rows


#### Μπορώ να καλώ τα dfs με τον παρακάτω τρόπο:

In [24]:
# 1ο sheet του excel: 1o item στη λίστα
# δηλαδή sheetsL[0] για το 1ο sheet, sheetsL[1] για το 2ο, κ.ο.κ.
sheetsL[0].head(20)

,English Word,Greek Word,Valence,Arousal,Dominance
0,aaaaaaah,ααααααχ,0.479,0.606,0.291
1,aaaah,ααααχ,0.520,0.636,0.282
2,aardvark,μυρμηγκοφάγο θηλαστικό,0.427,0.490,0.437
3,aback,προς τα πίσω,0.385,0.407,0.288
4,abacus,άβακας,0.510,0.276,0.485
5,abalone,αυτί της θάλασσας,0.500,0.480,0.412
6,abandon,εγκαταλείπω,0.052,0.519,0.245
7,abandoned,εγκαταλειμμένος,0.046,0.481,0.130
8,abandonment,εγκατάλειψη,0.128,0.430,0.202
9,abashed,σαν χαμένος,0.177,0.644,0.307


#### Τσεκάρω για missing values

In [25]:
sheetsL[0][sheetsL[0]['English Word'].isna()]

,English Word,Greek Word,Valence,Arousal,Dominance
11915,NaN,μηδενικό,0.253,0.147,0.152


In [26]:
sheetsL[0][sheetsL[0]['Greek Word'].isna()]

,English Word,Greek Word,Valence,Arousal,Dominance


In [27]:
sheetsL[0].loc[sheetsL[0]['English Word'].isna(), 'English Word'] = 'null'

#### Φέρνω και το csv με τις μεταφρασμένες λέξεις

In [28]:
translated_words_df = pd.read_csv('fixed_csvs/fixed_translated_words_2023_02.csv')
translated_words_df.head()

,English,Greek
0,abandon,εγκαταλείπω
1,abandoned,εγκαταλελειμμένο
2,abandonment,εγκατάλειψη
3,abase,παρακμή
4,abase,υποβαθμίζω


#### Ο παρακάτω κώδικας κάνει τα εξής:

- μετονομάζει τις στήλες "English Word" και "Greek Word", σε "English" και "Greek" αντίστοιχα
- βλέπει τις ελληνικές μεταφράσεις από το επιμελειμένο λεξικό και προσθέτει rows με τις επιπλέον μεταφράσεις

In [29]:
# φτιάχνω λίστα με τις μεταφρασμένες αγγλικές λέξεις
eng_wordsL = list(translated_words_df['English'].unique())
# φτιάχνω κενή λίστα για να προσθέτω τα νέα sheets
new_sheetsL = []
# για κάθε sheet του excel (το VAD έχει μόνο 1)
for sheet_df in sheetsL:
    # φτιάχνω κενή λίστα για να προσθέτω τα dfs με τα rows που θα προσθέσω
    temp_dfsL = []
    # κάνω αντίγραφο του sheet
    sheet_df_new = sheet_df.copy()
    # διορθώνω τα ονόματα των στηλών "English" και "Greek" αν δεν είναι σωστά
    for col in sheet_df_new.columns:
        if col.startswith('English'):
            sheet_df_new.rename({col: 'English'}, axis=1, inplace=True)
        if col.startswith('Greek'):
            # κάνω πεζά τις ελληνικές λέξεις
            sheet_df_new[col] = sheet_df_new[col].str.lower()
            sheet_df_new.rename({col: 'Greek'}, axis=1, inplace=True)
    # προσθέτω το διορθωμένο sheet στη λίστα που θα βάλω και τα dfs με τα rows που θα φτιάξω μετά
    temp_dfsL.append(sheet_df_new)
    # φτιάχνω λίστα με τις αγγλικές λέξεις του VAD
    vad_wL = list(sheet_df_new['English'].unique())
    # για κάθε αγγλική λέξη από τις επιμελειμένες μεταφράσεις 
    for w in eng_wordsL:
        # αν η λέξη είναι στο VAD
        if (w in vad_wL):
            # φτιάχνω μια λίστα με τις επιμελειμένες μεταφράσεις της
            transL = translated_words_df[translated_words_df['English'] == w]['Greek'].to_list()
            # μετράω πόσες είναι οι επιμελειμένες μεταφράσεις
            ntrans = len(transL)
            # εντοπίζω τη μετάφραση της αγγλικής λέξης στο VAD
            gw = sheet_df_new.loc[sheet_df_new['English']==w]['Greek'].item()
            # για όσες μεταφράσεις τις λέξεις υπάρχουν στο επιμελειμένο λεξικό
            for i in range(0, ntrans):
                # αν η μετάφραση από το επιμελειμένο λεξικό δεν είναι ίδια με τη μετάφραση στο VAD
                if gw != transL[i]:
                    # κάνε ένα αντίγραφο του row από το VAD
                    temp_df = sheet_df_new[sheet_df_new['English'] == w].copy()
                    # και βάλε στη στήλη "Greek" την επιπλέον μετάφραση
                    temp_df['Greek'] = transL[i]
                    # και πρόσθεσε το df με αυτό το νέο row στη λίστα 
                    temp_dfsL.append(temp_df)
    # ένωσε όλα τα dfs με τα παλιά και τα νέα rows
    new_sheet_df = pd.concat(temp_dfsL, ignore_index=True)
    # κάνε string τυχόν αριθμούς
    new_sheet_df['English'] = new_sheet_df['English'].astype('str')
    # ταξινόμησε το df ως προς τις αγγλικές λέξεις
    new_sheet_df = new_sheet_df.sort_values('English').reset_index(drop=True)
    # βάλε το νεο sheet στη λίστα των νέων sheets
    new_sheetsL.append(new_sheet_df)
# συνένωσε όλα τα sheets
greek_nrc_vad_lex_df = pd.concat(new_sheetsL, ignore_index=True)

greek_nrc_vad_lex_df

,English,Greek,Valence,Arousal,Dominance
0,False,ψευδής,0.120,0.404,0.240
1,True,αληθής,0.896,0.436,0.777
2,aaaaaaah,ααααααχ,0.479,0.606,0.291
3,aaaah,ααααχ,0.520,0.636,0.282
4,aardvark,μυρμηγκοφάγο θηλαστικό,0.427,0.490,0.437
...,...,...,...,...,...
25818,zoo,ζωολογικος κηπος,0.760,0.520,0.580
25819,zoological,ζωολογικός,0.667,0.458,0.492
25820,zoology,ζωολογία,0.568,0.347,0.509
25821,zoom,ανίπταμαι διαγωνίως,0.490,0.520,0.462


In [32]:
greek_nrc_vad_lex_df.tail(30)

,English,Greek,Valence,Arousal,Dominance
25793,zebra,ζέβρα,0.567,0.448,0.314
25794,zen,ζεν,0.492,0.210,0.432
25795,zenith,απόγειο,0.582,0.387,0.589
25796,zenith,ζενίθ,0.582,0.387,0.589
25797,zephyr,ζέφυρος,0.479,0.408,0.411
25798,zeppelin,ζέπελιν,0.542,0.550,0.472
25799,zest,ενθουσιασμός,0.917,0.769,0.845
25800,zest,όρεξη,0.917,0.769,0.845
25801,zest,ζέση,0.917,0.769,0.845
25802,zest,πάθος,0.917,0.769,0.845


In [33]:
greek_nrc_vad_lex_df['English'].nunique()

19971

#### Αποθήκευση του csv

In [34]:
# αποθήκευσε το λεξικό σαν csv
greek_nrc_vad_lex_df.to_csv('fixed_csvs/fixed_greek_NRC_VAD_lex_2023_01.csv', index=False)